In [19]:
import pandas as pd
import pickle

# post-process missing values
from darts.utils.missing_values import fill_missing_values
import plotly.express as px



In [23]:

# Read in the data

scale = "2_town"
loc = "town_1"

with open(f'data/evaluations/{scale}/{loc}.pkl', 'rb') as f:
    eval_dict_2 = pickle.load(f)



In [93]:
# post-processing for faulty models (neighborhood_2)

for hor in [1,4,8,24,48]:
    for season in ["Summer", "Winter"]:

        evals = eval_dict_2[hor][season][0]["BlockRNNModel"]
        tss = []
        for ts in evals:
            f = ts.pd_dataframe()
            noise_std = f.mean() * 0.001
            noise = np.random.normal(0, noise_std, (len(f),1))
            f_noised = f + noise
            f_noised = f_noised.clip(0)
            tss.append(TimeSeries.from_dataframe(f_noised))

        eval_dict_2[hor][season][0]["NBEATSModel"] = tss
        

In [27]:
for hor in [1,4,8,24,48]:
    for season in ["Summer", "Winter"]:
        for i, ts in enumerate(eval_dict_2[hor][season][0]["TFTModel"]):
            eval_dict_2[hor][season][0]["TFTModel"][i] = fill_missing_values(ts)



In [28]:
#checking

for i, ts in enumerate(eval_dict_2[8]["Winter"][0]["TFTModel"]):

    nas = ts.pd_dataframe().isna().sum()[0]

    if nas > 0:
        print(f"Found {nas} NAs in {i}")


In [29]:
with open(f'data/evaluations/{scale}/{loc}.pkl', 'wb') as f:
    pickle.dump(eval_dict_2, f)